Read cpu table from amazon.db

In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


db = sqlite3.connect('../data/amazon.db')

cpu_df = pd.read_sql('select * from cpu', db)
cpu_df.head()

,asin,catagory,manufacturer,brand,model_name,product_name,last_update
0,B07YP67PFV,cpu,Intel,Intel,core i9-10940X,Intel Core i9-10940X Desktop Processor 14 Core...,5616932
1,B07YP6Y9VY,cpu,Intel,Intel,core i9-10920X,Intel Core i9-10920X Desktop Processor 12 Core...,5616932
2,B07YP69HTM,cpu,Intel,Intel,core i9-10900X,Intel Core i9-10900X Desktop Processor 10 Core...,5616932
3,B07JGCMQY8,cpu,D&H,Intel,core i9-9980XE,Intel Core i9-9980XE Extreme Edition Processor...,5616904
4,B07JFH771Y,cpu,D&H,Intel,core i9-9940X,Intel Core i9-9940X X-Series Processor 14 Core...,5616932


Query the first asin in cpu_ts table with keepa_idx = 1.

In [5]:
intel_df = cpu_df[cpu_df['brand'] == 'Intel']
amd_df = cpu_df[cpu_df['brand'] == 'AMD']

AMD:
 - Zen2: 2019.5.27
 - Zen3: 2020.10.7
Intel:
 - 9th: 2019.4.23
 - 10th: 2020.4.30
 - 11th: 2021.3.15

In [31]:
def search(asin):
    cpu_ts = pd.read_sql_query(f'select keepa_minute, value1 from cpu_ts where asin = "{asin}" and keepa_idx=1', db)
    cpu_ts['ts'] = pd.to_datetime((cpu_ts['keepa_minute'] + 21564000) * 60, unit='s')
    cpu_ts.set_index('ts', inplace=True)
    cpu_ts = cpu_ts.resample('1D').ffill()
    cpu_ts = cpu_ts.resample('M').mean()
    ts_data = {}
    for d in ['2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12']:
        ts_data[d] = 1
    if '2020-04' in cpu_ts.index:
        base_price = cpu_ts['2020-04']['value1'][0]
        for date in ['2020-03', '2020-04', '2020-05', '2020-06', '2020-07']:
            ts_data[date] = cpu_ts[date]['value1'][0] / base_price if date in cpu_ts.index else 1
            if ts_data[date] < 0:
                ts_data[date] = 1
    if '2020-10' in cpu_ts.index:
        base_price = cpu_ts['2020-10']['value1'][0]
        for date in ['2020-08', '2020-09', '2020-10', '2020-11', '2020-12']:
            ts_data[date] = cpu_ts[date]['value1'][0] / base_price if date in cpu_ts.index else 1
            if ts_data[date] < 0:
                ts_data[date] = 1
    ts_data['Gen10Down'] = (ts_data['2020-06'] + ts_data['2020-07']) < (ts_data['2020-03'] + ts_data['2020-04'])
    ts_data['Zen3Down'] = (ts_data['2020-11'] + ts_data['2020-12']) < (ts_data['2020-08'] + ts_data['2020-09'])
    return ts_data

print(search(intel_df['asin'][1]))

{'2020-03': 1.1119349113942896, '2020-04': 1.0, '2020-05': 1.435418341152801, '2020-06': 1.2434503099717333, '2020-07': 1.0978144312263278, '2020-08': 1.1726182101550713, '2020-09': 0.9909796704350854, '2020-10': 1.0, '2020-11': 0.9682949109372571, '2020-12': 0.9387640249838345, 'Gen10Down': False, 'Zen3Down': True}


<ipython-input-31-0096a9b2665b>:11: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  base_price = cpu_ts['2020-04']['value1'][0]
<ipython-input-31-0096a9b2665b>:13: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  ts_data[date] = cpu_ts[date]['value1'][0] / base_price if date in cpu_ts.index else 1
<ipython-input-31-0096a9b2665b>:17: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  base_price = cpu_ts['2020-10']['value1'][0]
<ipython-input-31-0096a9b2665b>:19: FutureWarning: Indexing a DataFrame w

In [32]:
df_dict = []

for idx, row in intel_df.iterrows():
    ts = search(row['asin'])
    ts['brand'] = 'intel'
    ts['product'] = row['model_name']
    df_dict.append(ts)

for idx, row in amd_df.iterrows():
    ts = search(row['asin'])
    ts['brand'] = 'amd'
    ts['product'] = row['model_name']
    df_dict.append(ts)

df = pd.DataFrame(df_dict)

<ipython-input-31-0096a9b2665b>:11: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  base_price = cpu_ts['2020-04']['value1'][0]
<ipython-input-31-0096a9b2665b>:13: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  ts_data[date] = cpu_ts[date]['value1'][0] / base_price if date in cpu_ts.index else 1
<ipython-input-31-0096a9b2665b>:17: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  base_price = cpu_ts['2020-10']['value1'][0]
<ipython-input-31-0096a9b2665b>:19: FutureWarning: Indexing a DataFrame w

In [33]:
df.to_csv('ts.csv', index=False)

Close database connection.

In [34]:
db.close()